In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("spark://spark-master:7077")
         .appName("arrival")
         .getOrCreate())
sc = spark.sparkContext

22/11/03 08:01:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/03 08:01:52 WARN StandaloneAppClient$ClientEndpoint: Failed to connect to master spark-master:7077
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:101)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:109)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anon$1.run(StandaloneAppClient.scala:107)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java

22/11/03 08:02:32 WARN StandaloneAppClient$ClientEndpoint: Failed to connect to master spark-master:7077
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:101)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:109)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anon$1.run(StandaloneAppClient.scala:107)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)
Caused by: java.io.IOException: Fail

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalArgumentException: requirement failed: Can only call getServletHandlers on a running MetricsSystem
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.metrics.MetricsSystem.getServletHandlers(MetricsSystem.scala:92)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:597)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngi

In [ ]:
pyspark

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from glob import glob
import uuid

# Read CSV

In [3]:
glob('/home/jovyan/work/data/old/arrival_time/*.csv')

['/home/jovyan/work/data/old/arrival_time/869530043171871_6_go.csv',
 '/home/jovyan/work/data/old/arrival_time/864507039673982_35_back.csv',
 '/home/jovyan/work/data/old/arrival_time/868998030587691_56_back.csv',
 '/home/jovyan/work/data/old/arrival_time/869530043171863_6_back.csv',
 '/home/jovyan/work/data/old/arrival_time/868998030635219_133_go.csv',
 '/home/jovyan/work/data/old/arrival_time/867553050806694_56_go.csv',
 '/home/jovyan/work/data/old/arrival_time/868998030588186_133_go.csv',
 '/home/jovyan/work/data/old/arrival_time/868998030633206_56_back.csv',
 '/home/jovyan/work/data/old/arrival_time/866381052396907_133_go.csv',
 '/home/jovyan/work/data/old/arrival_time/866381052396311_6_back.csv',
 '/home/jovyan/work/data/old/arrival_time/864507039672166_6_go.csv',
 '/home/jovyan/work/data/old/arrival_time/866381052370274_133_go.csv',
 '/home/jovyan/work/data/old/arrival_time/868998030633305_56_back.csv',
 '/home/jovyan/work/data/old/arrival_time/867553050796408_56_go.csv',
 '/home/

In [5]:
df_arrival = (spark.read.format('csv')
              .option("header", True)
             .load("/home/jovyan/work/data/arrival/m04/arrival_time/*.csv"))
df_arrival = df_arrival.orderBy(df_arrival.time.asc())
df_arrival.printSchema()

root
 |-- station_lat: string (nullable = true)
 |-- station_long: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- time: string (nullable = true)
 |-- station_num: string (nullable = true)
 |-- loop_num: string (nullable = true)
 |-- path: string (nullable = true)
 |-- route_num: string (nullable = true)
 |-- gps_imei: string (nullable = true)



In [17]:
df_arrival = (df_arrival.withColumn("station_lat",col("station_lat").cast('double'))
              .withColumn("station_long", col("station_long").cast("double"))
              .withColumn("distance", col("distance").cast("double"))
              .withColumn("time", col("time").cast(StringType()))
              .withColumn("station_num", col("station_num").cast(IntegerType()))
              .withColumn("loop_num", col("loop_num").cast(IntegerType()))
              .withColumn("route_num", col("route_num").cast(IntegerType()))
              .withColumn("gps_imei", col("gps_imei").cast(LongType()))
             )
df_arrival.printSchema()

root
 |-- station_lat: double (nullable = true)
 |-- station_long: double (nullable = true)
 |-- distance: double (nullable = true)
 |-- time: string (nullable = true)
 |-- station_num: integer (nullable = true)
 |-- loop_num: integer (nullable = true)
 |-- path: string (nullable = true)
 |-- route_num: integer (nullable = true)
 |-- gps_imei: long (nullable = true)



In [18]:
class pythonFunctions:
    @udf
    def generate_uuid():
        return str(uuid.uuid4().hex)

In [19]:
df_arrival = df_arrival.withColumn("uuid", pythonFunctions.generate_uuid())

In [20]:
df_arrival.limit(5).toPandas()

,station_lat,station_long,distance,time,station_num,loop_num,path,route_num,gps_imei,uuid
0,13.659400,100.535141,3.399549,2022-03-22 08:03:15,0,0,go,6,864507039672166,ecca4ad39bbe4c1c9255145fe501f5ff
1,13.659541,100.533023,7.568617,2022-03-22 08:04:45,1,0,go,6,864507039672166,6e69e8029b9e4a4e8d031653eacd0f8a
2,13.660282,100.531442,2.932296,2022-03-22 08:05:45,2,0,go,6,864507039672166,cd0913aaa01f453bb0b37a19a8f52cf6
3,13.660546,100.529004,86.640243,2022-03-22 08:07:15,3,0,go,6,864507039672166,314cf278d5d44bc8b7da3bfcb8e8d69b
4,13.661442,100.526945,62.207002,2022-03-22 08:08:30,4,0,go,6,864507039672166,d19e78995c6046a99c758e1610422aa3


In [26]:
df_arrival.select('gps_imei').distinct().count()

19

# Upload to postgresql

In [98]:
(df_arrival.write
 .format("jdbc")
 .option("url", "jdbc:postgresql://postgres/test")
 .option("dbtable", "public.arrival_time")
 .option("user", "test")
 .option("password", "postgres")
 .mode("overwrite") # to append change to mode("append")
 .save())